In [ ]:
# Init Python

%load_ext autoreload
%reload_ext autoreload
%autoreload 2


print("Init Python")

import sys
import logging

# Add Subdirectory with custom Python source modules to path
sys.path.append("src")

# Log Level
# log_level = logging.INFO
log_level = logging.DEBUG

logger_name = "root"

logging.basicConfig(stream=sys.stdout, level=log_level)
log = logging.getLogger(logger_name)
log.setLevel(log_level)

http_log = logging.getLogger("urllib3.connectionpool")
http_log.setLevel(logging.INFO)

log.info("Successfully initialized Python")

In [ ]:
import csv
import pandas as pd
import numpy as np
import requests
import datetime

company = 'Example'

log.info("Download Google product categories")
url = 'http://www.google.com/basepages/producttype/taxonomy-with-ids.en-US.txt'
categoryfile = requests.get(url, allow_redirects=True)
open('data/taxonomy-with-ids.en-US.txt', 'wb').write(categoryfile.content)
with open("data/taxonomy-with-ids.en-US.txt", "rt") as fin:
    with open("data/taxonomy-with-ids.en-US.csv", "wt") as fout:
        for line in fin:
            fout.write(line.replace('# Google_Product_Taxonomy_Version: 2021-09-21','"google_category_id","google_category_name').replace(' - ', ',"').splitlines()[0] + '"\n')

log.info("Loading custom category mappings from CSV file")
google_category_mapping = pd.read_csv('input/google_category_mapping.csv', error_bad_lines=False, dtype = {
    'category': str,
    'sub_category': str,
    'sub_sub_category': str,
    'google_category_id': str,
    'google_category_name': str
})

log.info("Loading product CSV file(s)")
def read_csv(filelocation):
    return pd.read_csv(filelocation, encoding = "cp1252", error_bad_lines=False, dtype = {
        'SKU': str,
        'Title': str,
        'Description': str,
        'Detailed Description': str,
        'Thumbnail Image': str,
        'Product Image': str,
        'Price': np.float64,
        'Wholesale Price': str,
        'Weight (Kgs)': str,
        'Volume': np.int64,
        'Inactive': str,
        'Large Image': str,
        'GTIN': str,
        'Brand': str,
        'Manufacturer Part Number': str,
        'Product Page Title': str,
        'Product Keywords': str,
        'Product Page Description': str,
        'product_type': str,
        'Extended Code Field': str,
        'URL Override': str,
        'URL Slug': str,
        'Last Modification Date': str,
        'Supplier': str,
        'Custom Data 1': str,
        'Custom Data 2': str,
        'Custom Data 3': str,
        'Custom Data 4': str,
        'FAQs': str,
        'External Id': str,
        'frequency': str,
        'Quantity': str,
        'Stock Message': str,
        'Promotion Discount': str,
        'Promotion Type': str,
        'Promotion Text': str,
        'Promotion Timed': str,
        'Promotion Start Date': str,
        'Promotion End Date': str,
        'Product Tags': str,
        'Product Attributes': str,
        'Category': str,
        'Sub category': str,
        'Subx2 category': str,
        'Subx3 category': str,
        'Subx4 category': str
    })

page1 = read_csv('input/products_export_1-2000.csv')
page2 = read_csv('input/products_export_2001-4000.csv')
df1 = pd.concat([page1, page2])

log.info("Filter by active")
df2 = df1[df1['Inactive'] == '0']

log.info("Use only first category")
def map_reduce_to_first_cat(row):
    lines = str(row['Category']).splitlines()
    if(len(lines) > 1):
        return lines[0]
    return row['Category']

df2['Category'] = df2.apply(map_reduce_to_first_cat, axis=1)
def map_reduce_to_first_sub_cat(row):
    lines = str(row['Sub category']).splitlines()
    if(len(lines) > 1):
        return lines[0]
    return row['Sub category']

df2['Sub category'] = df2.apply(map_reduce_to_first_sub_cat, axis=1)

def map_reduce_to_first_sub_sub_cat(row):
    lines = str(row['Subx2 category']).splitlines()
    if(len(lines) > 1):
        return lines[0]
    return row['Subx2 category']

df2['Subx2 category'] = df2.apply(map_reduce_to_first_sub_sub_cat, axis=1)

log.info("Resolve Google category")
def map_google_category(row):
    cat_mapping = google_category_mapping[google_category_mapping['category'] == row['Category']]
    if(cat_mapping.empty):
        return google_category_mapping[google_category_mapping['category'] == 'DEFAULT']['google_category_name'].iloc[0]
    else:
        if(len(cat_mapping.index) == 1):
            return cat_mapping['google_category_name'].iloc[0]
        else:
            sub_cat_mapping = cat_mapping[cat_mapping['sub_category'] == row['Sub category']]
            if(sub_cat_mapping.empty):
                return cat_mapping['google_category_name'].iloc[0]
            else:
                if(len(sub_cat_mapping) == 1):
                    return sub_cat_mapping['google_category_name'].iloc[0]
                else:
                    sub_sub_cat_mapping = cat_mapping[cat_mapping['sub_sub_category'] == row['Subx2 category']]
                    if(sub_sub_cat_mapping.empty):
                        return sub_cat_mapping['google_category_name'].iloc[0]
                    else:
                        if(len(sub_sub_cat_mapping) == 1):
                            return sub_sub_cat_mapping['google_category_name'].iloc[0]

df2['google_product_category'] = df2.apply(map_google_category, axis=1)

log.info("Aggregate category to Google product type")
def map_google_product_type(row):
    cat = str(row['Category']).replace('\r', '').replace('\n', '')
    if(not str(row['Sub category']).replace('\r', '').replace('\n', '') == '' and not pd.isnull(row['Sub category'])):
        cat = cat + ' > ' + str(row['Sub category']).replace('\r', '').replace('\n', '')
        if(not str(row['Subx2 category']).replace('\r', '').replace('\n', '') == '' and not pd.isnull(row['Subx2 category'])):
            cat = cat + ' > ' + str(row['Subx2 category']).replace('\r', '').replace('\n', '')
            if(not str(row['Subx3 category']).replace('\r', '').replace('\n', '') == '' and not pd.isnull(row['Subx3 category'])):
                cat = cat + ' > ' + str(row['Subx3 category']).replace('\r', '').replace('\n', '')
                if(not str(row['Subx4 category']).replace('\r', '').replace('\n', '') == '' and not pd.isnull(row['Subx4 category'])):
                    cat = cat + ' > ' + str(row['Subx4 category']).replace('\r', '').replace('\n', '')
    return cat

df2['google_product_type'] = df2.apply(map_google_product_type, axis=1)

log.info("Full Image URL")
def map_image_link(row):
    if(pd.isnull(row['Large Image']) or str(row['Large Image']).replace('\r', '').replace('\n', '') == ''):
        return ''
    else:
        return str('https://www.' + company.lower() + '.co.nz/site/' + company.lower() + '/images/large/') + str(row['Large Image'])

df2['image_link'] = df2.apply(map_image_link, axis=1)

log.info("Rename Columns")
df2 = df2.rename(columns={'Title': 'title',
                    'Description': 'description',
                    'Brand': 'brand',
                   })
df3 = df2

log.info("Add additional columns")
df3['condition'] = 'new'
def map_mpn(row):
    return row['SKU'] if row['Manufacturer Part Number'] == '' else row['Manufacturer Part Number']

df3['mpn'] = df3.apply(map_mpn, axis=1)
df3['identifier_exists'] = 'no'
def map_availability2(row):
    return 'out_of_stock' if row['Stock Message'] == 'Out of Stock' else 'in_stock'

df3['availability'] = df3.apply(map_availability2, axis=1)
def map_availability1(row):
    return 'out_of_stock' if row['Quantity'] == '0' else row['availability']
#     return 'backorder' if row['Quantity'] == '0' else row['availability']

df3['availability'] = df3.apply(map_availability1, axis=1)

def map_price(row):
    return str(row['Price']) + ' NZD'

df3['price'] = df3.apply(map_price, axis=1)
def map_link2(row):
    return 'https://www.' + company.lower() + '.co.nz/product?sku=' + str(row['SKU'])

df3['link'] = df3.apply(map_link2, axis=1)
def map_brand(row):
    return company + ' Machine Tools' if row['brand'] == '' else row['brand']

df3['brand'] = df3.apply(map_brand, axis=1)
def map_sale_price(row):
    if(pd.isnull(row['Promotion Discount']) or str(row['Promotion Discount']).replace('\r', '').replace('\n', '') == '' or str(row['Promotion Discount']).replace('\r', '').replace('\n', '') == '0'):
        return ''
    else:
        if(pd.isnull(row['Promotion End Date']) or str(row['Promotion End Date']).replace('\r', '').replace('\n', '') == '' or str(row['Promotion End Date']).replace('\r', '').replace('\n', '') == '0'):
            return ''
        else:
            try:
                promotion_end_date = datetime.datetime.strptime(str(row['Promotion End Date']).replace('\r', '').replace('\n', ''), '%Y%m%d')
                now_date = datetime.datetime.now()
                if(now_date < promotion_end_date):
                    if '%' in str(row['Promotion Discount']):
                        return str(float(row['Price']) - (float(row['Price']) * (float(str(row['Promotion Discount']).replace('%', ''))/100))) + ' NZD'
                    elif str(row['Promotion Discount']).isnumeric():
                        return str(float(row['Price']) - float(row['Promotion Discount'])) + ' NZD'
                    else:
                        log.warning('Invalid Promotion Discount ' +  str(row['Promotion Discount']) + ' for ' + row['SKU'])
                        return ''
                else:
                    return ''
            except ValueError:
                log.warning('Invalid Promotion end date ' + str(row['Promotion End Date']) + ' for ' + row['SKU'])
                return ''

df3['sale_price'] = df3.apply(map_sale_price, axis=1)
def map_sale_price_effective_date(row):
    if(pd.isnull(row['sale_price']) or str(row['sale_price']).replace('\r', '').replace('\n', '') == '' or str(row['sale_price']).replace('\r', '').replace('\n', '') == '0'):
        return ''
    else:
        try:
            promotion_start_date = datetime.datetime.strptime(str(row['Promotion Start Date']).replace('\r', '').replace('\n', ''), '%Y%m%d')
            promotion_end_date = datetime.datetime.strptime(str(row['Promotion End Date']).replace('\r', '').replace('\n', ''), '%Y%m%d')
            return promotion_start_date.strftime('%Y-%m-%d') + 'T00:00Z/' + promotion_end_date.strftime('%Y-%m-%d') + 'T23:59Z'
        except:
            log.warn('Invalid Promotion date ' + str(row['Promotion End Date']) + ' for ' + row['SKU'])
            return ''

df3['sale_price_effective_date'] = df3.apply(map_sale_price_effective_date, axis=1)
df3['gender'] = 'unisex'
df3['age_group'] = 'adult'
def map_shipping_weight(row):
    if(pd.isnull(row['Weight (Kgs)']) or str(row['Weight (Kgs)']).replace('\r', '').replace('\n', '') == ''):
        return ''
    else:
        return str(row['Weight (Kgs)']) + ' kg'

df3['shipping_weight'] = df3.apply(map_shipping_weight, axis=1)
# df3['adult'] = str('false')
df3['adult'] = ''

# Empty columns
df3['promotion_id'] = ''
df3['item_group_id'] = ''
df3['is_bundle'] = ''
df3['custom_label_0'] = ''
df3['custom_label_1'] = ''
df3['shipping_label'] = ''
df3['size'] = ''
df3['color'] = ''
df3['size_type'] = ''
df3['additional_image_link'] = ''
df3['shipping_length'] = ''
df3['shipping_height'] = ''
df3['shipping_width'] = ''


log.info("Remove unused Columns")
df3 = df3.drop(columns=['Detailed Description',
                        'Thumbnail Image',
                        'Product Image',
                        'Price',
                        'Wholesale Price',
                        'Weight (Kgs)',
                        'Volume',
                        'Inactive',
                        'Large Image',
                        'GTIN',
                        'Manufacturer Part Number',
                        'Product Page Title',
                        'Product Keywords',
                        'Product Page Description',
                        'product_type',
                        'Extended Code Field',
                        'URL Override',
                        'URL Slug',
                        'Last Modification Date',
                        'Supplier',
                        'Custom Data 1',
                        'Custom Data 2',
                        'Custom Data 3',
                        'Custom Data 4',
                        'FAQs',
                        'External Id',
                        'frequency',
                        'Quantity',
                        'Stock Message',
                        'Promotion Discount',
                        'Promotion Type',
                        'Promotion Text',
                        'Promotion Timed',
                        'Promotion Start Date',
                        'Promotion End Date',
                        'Product Tags',
                        'Product Attributes',
                        'Category',
                        'Sub category',
                        'Subx2 category',
                        'Subx3 category',
                        'Subx4 category'])
df3 = df3.rename(columns={'SKU': 'id', 'google_product_type': 'product_type'})

df3.to_csv('output/products_google.csv', index=None)
df3.to_csv('output/products_facebook.csv', index=None)